In [1]:
import sys
sys.path.append('/home/jovyan/work') # here add notebooks if fails

In [2]:
from pyspark.sql import SparkSession
import nbimporter
from utils.vault_scripts import read_root_token, get_secret_from_vault
from pyspark.sql import functions as F
from pyspark.sql.functions import explode, col, when, lit, expr
from graphframes import GraphFrame

In [3]:
spark = SparkSession.builder.appName("ExpDataAnalysisExchangeRates").getOrCreate()

In [4]:
hadoopConf = spark._jsc.hadoopConfiguration()

In [5]:
AWS_KEY_ID = get_secret_from_vault("aws1", "keyid")
AWS_ACCESS_KEY = get_secret_from_vault("aws2", "accesskey")
AWS_S3_BUCKET = get_secret_from_vault("aws3", "s3bucket")

In [6]:
hadoopConf.set("fs.s3a.access.key", AWS_KEY_ID)
hadoopConf.set("fs.s3a.secret.key", AWS_ACCESS_KEY)
hadoopConf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

In [7]:
exchange_rate_ccix_path = f"s3a://{AWS_S3_BUCKET}/raw/exchange_rates_usd/*/*.json"

In [8]:
ex_ccix_df = spark.read.json(exchange_rate_ccix_path)

In [9]:
ex_ccix_df.printSchema()

root
 |-- CLOSE: double (nullable = true)
 |-- FIRST_MESSAGE_TIMESTAMP: long (nullable = true)
 |-- FIRST_MESSAGE_VALUE: double (nullable = true)
 |-- HIGH: double (nullable = true)
 |-- HIGH_MESSAGE_TIMESTAMP: long (nullable = true)
 |-- HIGH_MESSAGE_VALUE: double (nullable = true)
 |-- INSTRUMENT: string (nullable = true)
 |-- LAST_MESSAGE_TIMESTAMP: long (nullable = true)
 |-- LAST_MESSAGE_VALUE: double (nullable = true)
 |-- LOW: double (nullable = true)
 |-- LOW_MESSAGE_TIMESTAMP: long (nullable = true)
 |-- LOW_MESSAGE_VALUE: double (nullable = true)
 |-- MARKET: string (nullable = true)
 |-- OPEN: double (nullable = true)
 |-- QUOTE_VOLUME: double (nullable = true)
 |-- QUOTE_VOLUME_DIRECT: double (nullable = true)
 |-- QUOTE_VOLUME_TOP_TIER: double (nullable = true)
 |-- QUOTE_VOLUME_TOP_TIER_DIRECT: double (nullable = true)
 |-- TIMESTAMP: long (nullable = true)
 |-- TOTAL_INDEX_UPDATES: long (nullable = true)
 |-- TYPE: string (nullable = true)
 |-- UNIT: string (nullable = t